# Data & Analytics Challenge Fugo Case
---

Bu Case'de Fugo Games'e ait oyunlardan birinin kullanıcılarının 15 günlük metadata ve oyun içi davranış verisi verilmiştir. Target değer ise bu kullanıcıların oyundaki ilk 90 günlerinde getirdiği gelirdir.

Training seti 3 parçaya ayrılmış şekilde gelip bunlar *users_train.csv*, *user_features_train.csv* ve *targets_train.csv*'dir. Test seti de buna benzer olarak 2 parçaya ayrılmış olup target seti aranmaktadır.

Değerlendirme faktörü olarak RMSE (Root Mean Squared Error) kullanılmaktadır.

---

### Öncelikle importları, ayarları ve kullanacağımız sabitleri belirliyoruz

In [141]:
import pandas as pd
pd.options.display.max_columns = 100

import numpy as np

In [2]:
RANDOM_STATE = 42
N_JOBS = -1

### Şimdi sıra veri okumada

In [3]:
# ham değerler oldukları için değişken isimleri önemli değil
a = pd.read_csv('targets_train.csv')
b = pd.read_csv('user_features_test.csv')
c = pd.read_csv('user_features_train.csv')
d = pd.read_csv('users_test.csv')
e = pd.read_csv('users_train.csv')

a = pd.DataFrame(a)
b = pd.DataFrame(b)
c = pd.DataFrame(c)
d = pd.DataFrame(d)
e = pd.DataFrame(e)

In [4]:
# training seti için target özelliği ayırıyoruz
train_target = a.drop(columns=['ID'])
train_target

,TARGET
0,0.000000
1,0.018892
2,0.000000
3,0.046650
4,0.014680
...,...
878589,0.009567
878590,0.246248
878591,0.038446
878592,1.022793


In [5]:
# training seti birleştiriyoruz
train_set = pd.merge(e, c, on='ID')
train_set

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,RetentionD8,RetentionD9,RetentionD10,RetentionD11,RetentionD12,RetentionD13,RetentionD14,RetentionD15,LevelAdvancedCountD0,LevelAdvancedCountD1,LevelAdvancedCountD2,LevelAdvancedCountD3,LevelAdvancedCountD4,LevelAdvancedCountD5,LevelAdvancedCountD6,LevelAdvancedCountD7,LevelAdvancedCountD8,LevelAdvancedCountD9,LevelAdvancedCountD10,LevelAdvancedCountD11,LevelAdvancedCountD12,LevelAdvancedCountD13,LevelAdvancedCountD14,LevelAdvancedCountD15,Level_1_Duration,Level_2_Duration,Level_3_Duration,Level_4_Duration,Level_5_Duration,Level_6_Duration,Level_7_Duration,Level_8_Duration,Level_9_Duration,Level_10_Duration,AdRevenueD0,AdRevenueD1,AdRevenueD2,AdRevenueD3,AdRevenueD4,AdRevenueD5,AdRevenueD6,AdRevenueD7,AdRevenueD8,AdRevenueD9,AdRevenueD10,AdRevenueD11,AdRevenueD12,AdRevenueD13,AdRevenueD14,AdRevenueD15,IAPRevenueD0,IAPRevenueD1,IAPRevenueD2,IAPRevenueD3,IAPRevenueD4,IAPRevenueD5,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,unityads_int,3.314099,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0,69.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,applovin_int,1.681524,True,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,5,0,0,0,0,0,3,5,0,2,0,0,0,0,0,0,13.0,91.0,39.0,79.0,180.0,89.0,124.0,118.0,35.0,117.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.008674,0.000000,0.010218,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,applovin_int,10.718750,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,63.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,NaN,5.100000,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,19,8,9,0,0,0,0,0,0,0,0,0,0,0,0,0,23.0,141.0,131.0,118.0,77.0,107.0,77.0,182.0,42.0,156.0,0.002150,0.019159,0.025341,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,applovin_int,2.091409,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,52.0,30.0,84.0,139.0,96.0,268.0,97.0,44.0,122.0,0.014680,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [305]:
e.ad_network.value_counts()

ad_network
applovin_int         520202
unityads_int          40855
Facebook Ads           3643
googleadwords_int      3288
restricted              119
ironsource_int           13
Cross_sale                4
Name: count, dtype: int64

In [6]:
# test seti birleştiriyoruz
test_set = pd.merge(d, b, on='ID')
test_set

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,RetentionD8,RetentionD9,RetentionD10,RetentionD11,RetentionD12,RetentionD13,RetentionD14,RetentionD15,LevelAdvancedCountD0,LevelAdvancedCountD1,LevelAdvancedCountD2,LevelAdvancedCountD3,LevelAdvancedCountD4,LevelAdvancedCountD5,LevelAdvancedCountD6,LevelAdvancedCountD7,LevelAdvancedCountD8,LevelAdvancedCountD9,LevelAdvancedCountD10,LevelAdvancedCountD11,LevelAdvancedCountD12,LevelAdvancedCountD13,LevelAdvancedCountD14,LevelAdvancedCountD15,Level_1_Duration,Level_2_Duration,Level_3_Duration,Level_4_Duration,Level_5_Duration,Level_6_Duration,Level_7_Duration,Level_8_Duration,Level_9_Duration,Level_10_Duration,AdRevenueD0,AdRevenueD1,AdRevenueD2,AdRevenueD3,AdRevenueD4,AdRevenueD5,AdRevenueD6,AdRevenueD7,AdRevenueD8,AdRevenueD9,AdRevenueD10,AdRevenueD11,AdRevenueD12,AdRevenueD13,AdRevenueD14,AdRevenueD15,IAPRevenueD0,IAPRevenueD1,IAPRevenueD2,IAPRevenueD3,IAPRevenueD4,IAPRevenueD5,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,878594,2024-05-12,1715478163668000,1715467363668000,Argentina,Android,mobile,Motorola,Moto G32,False,applovin_int,1.444805,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,10,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,16.0,147.0,81.0,68.0,265.0,184.0,142.0,133.0,59.0,157.0,0.001595,0.009382,0.001740,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,878595,2024-01-26,1706254855890000,1706233255890000,Mexico,Android,mobile,OnePlus,Nord N20 SE,False,applovin_int,9.147972,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17.0,209.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,878596,2024-02-13,1707778260263000,1707781860263000,France,Android,mobile,Motorola,moto g13,False,applovin_int,40.731158,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,73.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,878597,2024-02-21,1708530744695000,1708519944695000,Brazil,Android,mobile,Samsung,Galaxy A03,False,applovin_int,4.967959,True,False,False,True,False,True,True,False,True,True,False,False,False,False,False,False,2,0,0,1,0,4,13,0,2,0,0,0,0,0,0,0,66.0,896.0,562.0,840.0,412.0,1001.0,530.0,536.0,85.0,562.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.156159,0.000000,0.112458,0.000451,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,878598,2024-05-03,1714706093405000,1714688093405000,Peru,Android,mobile,Xiaomi,Redmi 13C,False,applovin_int,2.445842,True,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,21,0,5,18,26,9,0,0,0,0,0,0,0,0,0,0,32.0,99.0,38.0,46.0,175.0,48.0,72.0,59.0,37.0,69.0,0.000000,0.000000,0.004202,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [7]:
# sonradan kullanmak için ID değerlerini ayırıyoruz
train_IDs = pd.DataFrame(e['ID'])
test_IDs = pd.DataFrame(d['ID'])

### Verileri inceleme

Verileri incelediğimiz zaman bu tahmin görevinde işimize yaramayacak, kullanmayacağımız özellikler görüyoruz. Bu özelliklere karar verirken kişisel sebepler (örneğin cihaz markasının herhangi bir etkisi olmadığını düşünmem gibi) ve herkesin hemfikir olabileceği (örneğin ID gibi her değeri eşsiz bir özelliğin katkı sağlamaktan öte bir kısayol tahminleyici olarak davranıp sonuçları kötü yönde manipüle edeceği gibi) sebepler etkili olmuştur.

Bazı özellikler de doğrudan kullanılmayacak olup sonraki aşamalarda feature engineering kısmı için saklanmıştır.

In [8]:
cols_to_drop = [
    'ID',
    'first_open_date',
    'local_first_open_timestamp',
    'country',
    'device_brand',
    'device_model',
    'has_ios_att_permission',
    'first_prediction'
]

In [9]:
train_set = train_set.drop(columns=cols_to_drop)
test_set = test_set.drop(columns=cols_to_drop)
train_set.shape

(878594, 78)

Feature engineering ile veri incelemesini beraber yürütüyoruz. Bu aşamada bazı özelliklerin toplamlarını kullanarak yeni birer özellik oluşturacağız. Çünkü bu özelliklerin toplamları bile kendi başına apayrı bir bilgilendirici durumunda.

In [10]:
def create_sum_feature(df, cols, new_col):
    df[new_col] = df[cols].sum(axis=1)
    df = df.drop(columns=cols)
    return df

In [11]:
retention_cols = [
    'RetentionD0',
    'RetentionD1',
    'RetentionD2',
    'RetentionD3',
    'RetentionD4',
    'RetentionD5',
    'RetentionD6',
    'RetentionD7',
    'RetentionD8',
    'RetentionD9',
    'RetentionD10',
    'RetentionD11',
    'RetentionD12',
    'RetentionD13',
    'RetentionD14',
    'RetentionD15'
]

level_advanced_cols = [
    'LevelAdvancedCountD0',
    'LevelAdvancedCountD1',
    'LevelAdvancedCountD2',
    'LevelAdvancedCountD3',
    'LevelAdvancedCountD4',
    'LevelAdvancedCountD5',
    'LevelAdvancedCountD6',
    'LevelAdvancedCountD7',
    'LevelAdvancedCountD8',
    'LevelAdvancedCountD9',
    'LevelAdvancedCountD10',
    'LevelAdvancedCountD11',
    'LevelAdvancedCountD12',
    'LevelAdvancedCountD13',
    'LevelAdvancedCountD14',
    'LevelAdvancedCountD15'
]

level_duration_cols = [
    'Level_1_Duration',
    'Level_2_Duration',
    'Level_3_Duration',
    'Level_4_Duration',
    'Level_5_Duration',
    'Level_6_Duration',
    'Level_7_Duration',
    'Level_8_Duration',
    'Level_9_Duration',
    'Level_10_Duration'
]

ad_revenue_cols = [
    'AdRevenueD0',
    'AdRevenueD1',
    'AdRevenueD2',
    'AdRevenueD3',
    'AdRevenueD4',
    'AdRevenueD5',
    'AdRevenueD6',
    'AdRevenueD7',
    'AdRevenueD8',
    'AdRevenueD9',
    'AdRevenueD10',
    'AdRevenueD11',
    'AdRevenueD12',
    'AdRevenueD13',
    'AdRevenueD14',
    'AdRevenueD15'
]

iap_revenue_cols = [ 
    'IAPRevenueD0',
    'IAPRevenueD1',
    'IAPRevenueD2',
    'IAPRevenueD3',
    'IAPRevenueD4',
    'IAPRevenueD5',
    'IAPRevenueD6',
    'IAPRevenueD7',
    'IAPRevenueD8',
    'IAPRevenueD9',
    'IAPRevenueD10',
    'IAPRevenueD11',
    'IAPRevenueD12',
    'IAPRevenueD13',
    'IAPRevenueD14',
    'IAPRevenueD15'
]

In [12]:
# burada oluşturduğumuz setler yukarıdaki özelliklerin sadece toplam değerlerine sahip olacak, deneme amaçlı
train_set_sum_only = train_set.copy()
test_set_sum_only = test_set.copy()

In [13]:
train_set_sum_only = create_sum_feature(train_set_sum_only, retention_cols, 'Retention_Sum')
train_set_sum_only = create_sum_feature(train_set_sum_only, level_advanced_cols, 'Level_Advanced_Sum')
train_set_sum_only = create_sum_feature(train_set_sum_only, level_duration_cols, 'Level_Duration_Sum')
train_set_sum_only = create_sum_feature(train_set_sum_only, ad_revenue_cols, 'Ad_Revenue_Sum')
train_set_sum_only = create_sum_feature(train_set_sum_only, iap_revenue_cols, 'IAP_Revenue_Sum')

test_set_sum_only = create_sum_feature(test_set_sum_only, retention_cols, 'Retention_Sum')
test_set_sum_only = create_sum_feature(test_set_sum_only, level_advanced_cols, 'Level_Advanced_Sum')
test_set_sum_only = create_sum_feature(test_set_sum_only, level_duration_cols, 'Level_Duration_Sum')
test_set_sum_only = create_sum_feature(test_set_sum_only, ad_revenue_cols, 'Ad_Revenue_Sum')
test_set_sum_only = create_sum_feature(test_set_sum_only, iap_revenue_cols, 'IAP_Revenue_Sum')

In [14]:
# oluşturduğumuz seti biraz inceleyelim
train_set_sum_only

,first_open_timestamp,platform,device_category,ad_network,Retention_Sum,Level_Advanced_Sum,Level_Duration_Sum,Ad_Revenue_Sum,IAP_Revenue_Sum
0,1709355895042000,Android,mobile,unityads_int,2,3,131.0,0.000000,0.0
1,1710824539731000,Android,mobile,applovin_int,4,15,885.0,0.018892,0.0
2,1710731043082000,Android,mobile,applovin_int,1,3,157.0,0.000000,0.0
3,1709455862260000,iOS,mobile,NaN,3,36,1054.0,0.046650,0.0
4,1714482477190000,Android,mobile,applovin_int,1,15,940.0,0.014680,0.0
...,...,...,...,...,...,...,...,...,...
878589,1714180226014000,Android,mobile,applovin_int,2,10,1887.0,0.009567,0.0
878590,1710548138205000,Android,mobile,NaN,9,28,4743.0,0.135824,0.0
878591,1710283704357000,Android,mobile,applovin_int,1,19,912.0,0.038446,0.0
878592,1713633879783000,iOS,mobile,NaN,10,85,1384.0,0.277735,0.0


Şu haliyle kullanamayacağımız pek çok özellik var, hem sadece toplam özellikleri barındıran sette hem de diğerinde. Encoding işlemlerine geçiyoruz.

In [15]:
train_set.platform.value_counts()

platform
Android    514971
iOS        363623
Name: count, dtype: int64

In [16]:
platform_d = {
    'Android': 1,
    'iOS': 0
}

In [17]:
train_set['platform'] = train_set['platform'].map(platform_d)
train_set_sum_only['platform'] = train_set_sum_only['platform'].map(platform_d)
test_set['platform'] = test_set['platform'].map(platform_d)
test_set_sum_only['platform'] = test_set_sum_only['platform'].map(platform_d)

In [18]:
train_set.device_category.value_counts()

device_category
mobile    794214
tablet     84380
Name: count, dtype: int64

In [19]:
device_category_d = {
    'mobile': 1,
    'tablet': 0
}

In [20]:
train_set['device_category'] = train_set['device_category'].map(device_category_d)
train_set_sum_only['device_category'] = train_set_sum_only['device_category'].map(device_category_d)
test_set['device_category'] = test_set['device_category'].map(device_category_d)
test_set_sum_only['device_category'] = test_set_sum_only['device_category'].map(device_category_d)

*ad_network* özelliğine geldiğimizde işler biraz değişiyor. Verinin açıklamasını incelediğimizde burada null girdiler beklememiz gerektiğini anlıyoruz. Null veri olup olmadığını inceleyeceğiz öncelikle.

In [21]:
train_set.ad_network.isnull().sum()

310470

Evet, var. Null değerleri de encodelamamız gerekiyor. Bu değerleri 0 ile doldurup encode işlemine devam ediyoruz.

Ayrıca diğer özelliklerdeki muhtemel null değerlerini de doldurmak için (onları da 0 ile doldurmak da sorun yaratmayacağı için) tüm veri genelinde bir işlem yapıyoruz.

In [22]:
train_set = train_set.fillna(0)
train_set_sum_only = train_set_sum_only.fillna(0)
test_set = test_set.fillna(0)
test_set_sum_only = test_set_sum_only.fillna(0)

In [23]:
ad_network_d = {
    0: 0,
    'restricted': 1,
    'Cross_sale': 2,
    'ironsource_int': 3,
    'googleadwords_int': 4,
    'Facebook Ads': 5,
    'unityads_int': 6,
    'applovin_int': 7
}

In [24]:
train_set['ad_network'] = train_set['ad_network'].map(ad_network_d)
train_set_sum_only['ad_network'] = train_set_sum_only['ad_network'].map(ad_network_d)
test_set['ad_network'] = test_set['ad_network'].map(ad_network_d)
test_set_sum_only['ad_network'] = test_set_sum_only['ad_network'].map(ad_network_d)

Veride bazı girdiler True/False şeklindeydi. Bunları 1/0 şekline çevirmemiz gerekiyor.

In [25]:
boolean_columns = train_set.select_dtypes(include='bool').columns
train_set[boolean_columns] = train_set[boolean_columns].astype(int)

boolean_columns = train_set_sum_only.select_dtypes(include='bool').columns
train_set_sum_only[boolean_columns] = train_set_sum_only[boolean_columns].astype(int)

boolean_columns = test_set.select_dtypes(include='bool').columns
test_set[boolean_columns] = test_set[boolean_columns].astype(int)

boolean_columns = test_set_sum_only.select_dtypes(include='bool').columns
test_set_sum_only[boolean_columns] = test_set_sum_only[boolean_columns].astype(int)

In [26]:
test_set

,first_open_timestamp,platform,device_category,ad_network,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,RetentionD8,RetentionD9,RetentionD10,RetentionD11,RetentionD12,RetentionD13,RetentionD14,RetentionD15,LevelAdvancedCountD0,LevelAdvancedCountD1,LevelAdvancedCountD2,LevelAdvancedCountD3,LevelAdvancedCountD4,LevelAdvancedCountD5,LevelAdvancedCountD6,LevelAdvancedCountD7,LevelAdvancedCountD8,LevelAdvancedCountD9,LevelAdvancedCountD10,LevelAdvancedCountD11,LevelAdvancedCountD12,LevelAdvancedCountD13,LevelAdvancedCountD14,LevelAdvancedCountD15,Level_1_Duration,Level_2_Duration,Level_3_Duration,Level_4_Duration,Level_5_Duration,Level_6_Duration,Level_7_Duration,Level_8_Duration,Level_9_Duration,Level_10_Duration,AdRevenueD0,AdRevenueD1,AdRevenueD2,AdRevenueD3,AdRevenueD4,AdRevenueD5,AdRevenueD6,AdRevenueD7,AdRevenueD8,AdRevenueD9,AdRevenueD10,AdRevenueD11,AdRevenueD12,AdRevenueD13,AdRevenueD14,AdRevenueD15,IAPRevenueD0,IAPRevenueD1,IAPRevenueD2,IAPRevenueD3,IAPRevenueD4,IAPRevenueD5,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,1715478163668000,1,1,7,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,10,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,16.0,147.0,81.0,68.0,265.0,184.0,142.0,133.0,59.0,157.0,0.001595,0.009382,0.001740,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1706254855890000,1,1,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17.0,209.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1707778260263000,1,1,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,73.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1708530744695000,1,1,7,1,0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,2,0,0,1,0,4,13,0,2,0,0,0,0,0,0,0,66.0,896.0,562.0,840.0,412.0,1001.0,530.0,536.0,85.0,562.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.156159,0.000000,0.112458,0.000451,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1714706093405000,1,1,7,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,21,0,5,18,26,9,0,0,0,0,0,0,0,0,0,0,32.0,99.0,38.0,46.0,175.0,48.0,72.0,59.0,37.0,69.0,0.000000,0.000000,0.004202,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585725,1707496901535000,1,1,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17.0,63.0,168.0,174.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
585726,1714017577879000,0,1,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.0,16.0,56.0,40.0,16.0,20.0,80.0,43.0,40.0,58.0,3.500052,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
585727,1707930024996000,0,1,7,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,14,8,6,0,0,0,0,0,0,0,0,0,9,0,0,0,6.0,9.0,24.0,27.0,17.0,15.0,23.0,12.0,31.0,56.0,0.422472,0.407156,0.215020,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.315011,0.0,0.0,0.0,0.0,0.0,0.0,

Şimdi sıra geldi *first_open_timestamp* özelliğini kullanmaya. Bu özellik şu anki haliyle hiçbir şey ifade etmiyor. Fakat düşünürsek bir kullanıcının oyuna başlama tarihi ne kadar eskiyse o kadar fazla gelir getirmesi beklenir. Oyunun çıkış tarihine erişme imkanımız olmadığı için elimizdeki verideki ilk oynama tarihini görece başlangıç değeri olarak alıp kullanıcıların bu değere göre oyuna başlama tarihlerini kullanarak bir özellik oluşturuyoruz.

In [27]:
first_open_train = e.first_open_timestamp
first_open_test = d.first_open_timestamp

In [28]:
max_first_open_train = max(first_open_train)
max_first_open_test = max(first_open_test)

In [29]:
train_set['Relative_First_Open'] = max_first_open_train - first_open_train
train_set_sum_only['Relative_First_Open'] = max_first_open_train - first_open_train
test_set['Relative_First_Open'] = max_first_open_test - first_open_test
test_set_sum_only['Relative_First_Open'] = max_first_open_test - first_open_test

In [30]:
train_set = train_set.drop(columns=['first_open_timestamp'])
train_set_sum_only = train_set_sum_only.drop(columns=['first_open_timestamp'])
test_set = test_set.drop(columns=['first_open_timestamp'])
test_set_sum_only = test_set_sum_only.drop(columns=['first_open_timestamp'])

Şimdi model denemeye geçmeden önce çeşitlilik amacıyla hem oluşturduğumuz toplam özelliklere hem de bu özellikleri oluşturan tekil özelliklere sahip bir veri daha oluşturuyoruz.

In [31]:
train_set_both = train_set.copy()
train_set_both['Retention_Sum'] = train_set_sum_only['Retention_Sum']
train_set_both['Level_Advanced_Sum'] = train_set_sum_only['Level_Advanced_Sum']
train_set_both['Level_Duration_Sum'] = train_set_sum_only['Level_Duration_Sum']
train_set_both['Ad_Revenue_Sum'] = train_set_sum_only['Ad_Revenue_Sum']
train_set_both['IAP_Revenue_Sum'] = train_set_sum_only['IAP_Revenue_Sum']

In [32]:
test_set_both = test_set.copy()
test_set_both['Retention_Sum'] = test_set_sum_only['Retention_Sum']
test_set_both['Level_Advanced_Sum'] = test_set_sum_only['Level_Advanced_Sum']
test_set_both['Level_Duration_Sum'] = test_set_sum_only['Level_Duration_Sum']
test_set_both['Ad_Revenue_Sum'] = test_set_sum_only['Ad_Revenue_Sum']
test_set_both['IAP_Revenue_Sum'] = test_set_sum_only['IAP_Revenue_Sum']

### Model deneme

Bu bir regresyon problemi olduğu için öncelikle hazır regresyon modellerini deniyoruz.

In [33]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoLarsIC, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, ARDRegression, BayesianRidge, PassiveAggressiveRegressor, HuberRegressor, TweedieRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

Split ile kullanmak için elimizde tam bir test verisi yok, o yüzden elimizdeki training verisini ikiye ayırarak bu veri içinde bir train ve test veri çifti oluşturacağız.

In [34]:
metrics = {}

# denemek için modeller
model_dict = {
    'LinearRegression': LinearRegression(n_jobs=N_JOBS),
    'Ridge': Ridge(random_state=RANDOM_STATE),
    'Lasso': Lasso(random_state=RANDOM_STATE),
    'LassoLarsIC': LassoLarsIC(),
    'ElasticNet': ElasticNet(),
    'Lars': Lars(random_state=RANDOM_STATE),
    'LassoLars': LassoLars(random_state=RANDOM_STATE),
    'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit(),
    'ARDRegression': ARDRegression(),
    'BayesianRidge': BayesianRidge(),
    'PassiveAggressiveRegressor': PassiveAggressiveRegressor(random_state=RANDOM_STATE),
    'HuberRegressor': HuberRegressor(),
    'TweedieRegressor': TweedieRegressor()
}

In [35]:
# model değerlendirme fonksiyonu
def model_eval(model_dict, X_train, y_train, X_test, y_test):

    for model_name, model in model_dict.items():
        print(f'Model: {model_name}')

        model.fit(X_train, y_train)

        predictions = model.predict(X_test)

        rmse = root_mean_squared_error(y_test, predictions)

        print(f'RMSE: {rmse:.6f}')

        metrics[model_name] = rmse
        print('\n'+'='*50+'\n')

    d = dict(sorted(metrics.items(), key=lambda item: item[1]))

    print('Scoreboard:')

    for model_name, rmse in d.items():
        print(f'{model_name}: {rmse}')

In [36]:
# dummy suffixi bir nevi pseudo-data oluşturduğumu belirtmek için kendimce koyduğum bir not
# gözümde bu iki veri diğerinden daha ham olduğundan dolayı ileride deneme için kullanmak istersem diye bir kopyasını alıyorum
train_set_dummy = train_set.copy()
train_set_sum_only_dummy = train_set_sum_only.copy()

In [37]:
X_dummy_train, X_dummy_test, y_dummy_train, y_dummy_test = train_test_split(
    train_set_dummy, train_target, test_size=.3, random_state=RANDOM_STATE)

X_dummy_sum_train, X_dummy_sum_test, y_dummy_sum_train, y_dummy_sum_test = train_test_split(
    train_set_sum_only_dummy, train_target, test_size=.3, random_state=RANDOM_STATE)

X_dummy_both_train, X_dummy_both_test, y_dummy_both_train, y_dummy_both_test = train_test_split(
    train_set_both, train_target, test_size=.3, random_state=RANDOM_STATE)

### Toplam özelliği olmayan veri ile test

relative zımbırtıyı koydum diğerinden farklı olarak, rmse biraz arttı

In [38]:
model_eval(model_dict, X_dummy_train, y_dummy_train, X_dummy_test, y_dummy_test)

Model: LinearRegression
RMSE: 1.479048


Model: Ridge


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.28604e-31): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


RMSE: 1.453886


Model: Lasso
RMSE: 2.720929


Model: LassoLarsIC


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 1.453953


Model: ElasticNet
RMSE: 2.702881


Model: Lars
RMSE: 1.634787


Model: LassoLars
RMSE: 2.720929


Model: OrthogonalMatchingPursuit
RMSE: 2.850238


Model: ARDRegression


...

RMSE: 2.879855


Model: HuberRegressor


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.811085


Model: TweedieRegressor


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.792711


Scoreboard:
Ridge: 1.4538863405867446
LassoLarsIC: 1.4539530085344923
BayesianRidge: 1.4543367967931642
LinearRegression: 1.4790480346130515
Lars: 1.6347869739501604
ElasticNet: 2.7028813394683358
LassoLars: 2.7209286663372176
Lasso: 2.720928685317501
ARDRegression: 2.792702018348974
TweedieRegressor: 2.7927112222217874
HuberRegressor: 2.8110849853047233
OrthogonalMatchingPursuit: 2.8502380873890716
PassiveAggressiveRegressor: 2.879855019350079


### Toplam özelliği olan veri ile test

In [39]:
metrics = {}
model_eval(model_dict, X_dummy_sum_train, y_dummy_sum_train, X_dummy_sum_test, y_dummy_sum_test)

Model: LinearRegression
RMSE: 1.526062


Model: Ridge
RMSE: 1.526062


Model: Lasso


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.19678e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 1.706211


Model: LassoLarsIC
RMSE: 1.526062


Model: ElasticNet
RMSE: 1.710160


Model: Lars
RMSE: 1.527186


Model: LassoLars
RMSE: 1.706208


Model: OrthogonalMatchingPursuit
RMSE: 2.792573


Model: ARDRegression
RMSE: 2.792702


Model: BayesianRidge


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 1.526061


Model: PassiveAggressiveRegressor


...

RMSE: 2.904267


Model: HuberRegressor


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.811085


Model: TweedieRegressor


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.792711


Scoreboard:
BayesianRidge: 1.526060785773913
Ridge: 1.526062001285606
LassoLarsIC: 1.526062360894074
LinearRegression: 1.5260623986326847
Lars: 1.5271856339646
LassoLars: 1.7062082113517205
Lasso: 1.7062113503226217
ElasticNet: 1.7101595396319837
OrthogonalMatchingPursuit: 2.79257254463364
ARDRegression: 2.792702018348974
TweedieRegressor: 2.792711222221788
HuberRegressor: 2.8110849853047233
PassiveAggressiveRegressor: 2.9042666545221922


### İki tür özelliği de barındıran veri ile test

In [40]:
metrics = {}
model_eval(model_dict, X_dummy_both_train, y_dummy_both_train, X_dummy_both_test, y_dummy_both_test)

Model: LinearRegression
RMSE: 1.452981


Model: Ridge


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.28573e-31): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


RMSE: 1.453901


Model: Lasso
RMSE: 1.708230


Model: LassoLarsIC


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 1.453953


Model: ElasticNet


...

RMSE: 2.900441


Model: HuberRegressor


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.811085


Model: TweedieRegressor


c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.792711


Scoreboard:
BayesianRidge: 1.4528940128899959
LinearRegression: 1.4529811527997276
Ridge: 1.453900932360117
LassoLarsIC: 1.4539530085322097
LassoLars: 1.708229676951904
Lasso: 1.7082296769519087
ElasticNet: 1.7110076245715604
OrthogonalMatchingPursuit: 2.7202133409607003
ARDRegression: 2.792702018348974
TweedieRegressor: 2.792711222221788
HuberRegressor: 2.8110849853047233
PassiveAggressiveRegressor: 2.9004406651127943
Lars: 21.442959708303615


Sonuçların tutarlı olanlarını incelediğimizde iki türdeki özelliği de barındıran veriyi yeğlemeye karar veriyoruz. Yine sonuçlara ve algoritmaları inceleyince fine-tune etmek için 3 algoritmaya karar veriyorum.

### Fine-tuning

In [41]:
from skopt import BayesSearchCV

In [42]:
# algoritmaları matematiksel olarak inceleyip kullandığı parametrelere uygun bulduğum aralık ve değerleri tanımlıyorum
param_spaces = {
    'ARDRegression': {
        'max_iter': [300, 500],
        'tol': [0.001, 0.01, 0.0001],
        'alpha_1': [1e-06, 1e-04, 1e-02],
        'alpha_2': [1e-06, 1e-04, 1e-02],
        'lambda_1': [1e-06, 1e-04, 1e-02],
        'lambda_2' : [1e-06, 1e-04, 1e-02]
    },

    'BayesianRidge': {
        'max_iter': [300, 500],
        'tol': [0.001, 0.01, 0.0001],
        'alpha_1': [1e-06, 1e-04, 1e-02],
        'alpha_2': [1e-06, 1e-04, 1e-02],
        'lambda_1': [1e-06, 1e-04, 1e-02],
        'lambda_2' : [1e-06, 1e-04, 1e-02]
    },

    'LassoLarsIC': {
        'criterion': ['aic', 'bic'],
        'max_iter': [300, 500]
    }
}

In [147]:
# fine-tuning ve değerlendirme için fonksiyonlar
def param_finder(model, X_train, y_train):
    param_space = param_spaces[model.__class__.__name__]
    model = globals()[model.__class__.__name__]()

    bayes_search = BayesSearchCV(
        model, 
        param_space, 
        n_jobs=N_JOBS, 
        cv=5, 
        n_iter=20, 
        random_state=RANDOM_STATE
    )
    
    bayes_search.fit(X_train, y_train)

    return bayes_search.best_params_


def metric_eval(y_test, y_pred):
    rmse = root_mean_squared_error(y_test, y_pred)

    print(f"RMSE: {round(rmse, 6)}")


def prime_model_eval(model, X_train, X_test, y_train, y_test):
    params = param_finder(model, X_train, y_train)

    print(f"{model.__class__.__name__} için en iyi parametreler:")
    print(params)
    print()

    model = globals()[model.__class__.__name__](**params)
        
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    metric_eval(y_test, y_pred)

In [44]:
prime_model_eval(ARDRegression(), X_dummy_both_train, X_dummy_both_test, y_dummy_both_train, y_dummy_both_test)

c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=300, high=500, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(300, 500), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=300, high=500, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(300, 500), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=

ARDRegression için en iyi parametreler:
OrderedDict([('alpha_1', 0.01), ('alpha_2', 0.01), ('lambda_1', 1e-06), ('lambda_2', 0.01), ('max_iter', 473), ('tol', 0.001)])



c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 2.792573


In [45]:
prime_model_eval(BayesianRidge(), X_dummy_both_train, X_dummy_both_test, y_dummy_both_train, y_dummy_both_test)

c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=300, high=500, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(300, 500), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=300, high=500, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(300, 500), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=

BayesianRidge için en iyi parametreler:
OrderedDict([('alpha_1', 1e-06), ('alpha_2', 0.01), ('lambda_1', 0.01), ('lambda_2', 1e-06), ('max_iter', 304), ('tol', 0.01)])



c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 1.452894


In [46]:
prime_model_eval(LassoLarsIC(), X_dummy_both_train, X_dummy_both_test, y_dummy_both_train, y_dummy_both_test)

c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=300, high=500, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(300, 500), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=300, high=500, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(300, 500), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [300, 500] was inferred to Integer(low=

LassoLarsIC için en iyi parametreler:
OrderedDict([('criterion', 'bic'), ('max_iter', 477)])



c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 1.453292


Bundan sonrası çeşitli kombinasyonlarla tahminler oluşturup kaggle'a submission atmakta. Deney aşamaları çok karışık olduğu için başarısız denemeleri sildim.

In [294]:
model = BayesianRidge(alpha_1=0.0001, alpha_2=0.01, lambda_1=0.01, lambda_2=1e-06, max_iter=434, tol=0.01)

In [48]:
model.fit(train_set_both, train_target)
yhat = model.predict(test_set_both)

c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
yhat

array([ 1.26514048e-02,  2.15921675e-02,  1.97335385e-02, ...,
        1.85869605e+00, -2.05396461e-04,  1.21583792e+00])

In [50]:
submission = test_IDs.copy()
submission['TARGET'] = yhat

In [51]:
submission

,ID,TARGET
0,878594,0.012651
1,878595,0.021592
2,878596,0.019734
3,878597,0.409403
4,878598,0.037530
...,...,...
585725,1464319,0.021952
585726,1464320,3.994350
585727,1464321,1.858696
585728,1464322,-0.000205


In [52]:
submission.to_csv('submission.csv', index=False)

Şimdi durup biraz düşünme vakti. Bizim tahmin ettiğimiz özellik bir gelir belirtiyor. Bu özellik negatif değerlere sahip olamaz. Bunu göz önünde bulundurarak negatif tahminleri 0'a clipliyoruz.

In [53]:
submission.loc[submission['TARGET'] < 0, 'TARGET'] = 0

In [54]:
submission.to_csv('submission_final.csv', index=False)

---

In [247]:
exp_train_set_both = train_set_both.copy()
exp_test_set_both = test_set_both.copy()

In [248]:
d

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network
0,878594,2024-05-12,1715478163668000,1715467363668000,Argentina,Android,mobile,Motorola,Moto G32,False,applovin_int
1,878595,2024-01-26,1706254855890000,1706233255890000,Mexico,Android,mobile,OnePlus,Nord N20 SE,False,applovin_int
2,878596,2024-02-13,1707778260263000,1707781860263000,France,Android,mobile,Motorola,moto g13,False,applovin_int
3,878597,2024-02-21,1708530744695000,1708519944695000,Brazil,Android,mobile,Samsung,Galaxy A03,False,applovin_int
4,878598,2024-05-03,1714706093405000,1714688093405000,Peru,Android,mobile,Xiaomi,Redmi 13C,False,applovin_int
...,...,...,...,...,...,...,...,...,...,...,...
585725,1464319,2024-02-09,1707496901535000,1707486101535000,Argentina,Android,mobile,Motorola,Moto G52,False,applovin_int
585726,1464320,2024-04-25,1714017577879000,1713992377879000,United States,iOS,mobile,Apple,iPhone 14 Plus,True,applovin_int
585727,1464321,2024-02-14,1707930024996000,1707912024996000,United States,iOS,mobile,Apple,iPhone 11,False,applovin_int
585728,1464322,2024-05-07,1715043787322000,1715032987322000,Argentina,Android,mobile,Samsung,Galaxy A22 4G,False,applovin_int


In [249]:
e

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,unityads_int
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,applovin_int
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,applovin_int
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,NaN
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,applovin_int
...,...,...,...,...,...,...,...,...,...,...,...
878589,878589,2024-04-27,1714180226014000,1714158626014000,Mexico,Android,mobile,Xiaomi,Redmi 9A,False,applovin_int
878590,878590,2024-03-16,1710548138205000,1710537338205000,Argentina,Android,mobile,Motorola,Moto E7i Power,False,NaN
878591,878591,2024-03-13,1710283704357000,1710290904357000,Finland,Android,mobile,Motorola,moto g51 5G,False,applovin_int
878592,878592,2024-04-20,1713633879783000,1713641079783000,Italy,iOS,mobile,Apple,iPhone 14 Pro Max,False,NaN


In [250]:
# copying platform, device_category and ad_network columns from old sets to new exp sets
exp_train_set_both['platform'] = e['platform']
exp_train_set_both['device_category'] = e['device_category']
exp_train_set_both['ad_network'] = e['ad_network']

exp_test_set_both['platform'] = d['platform']
exp_test_set_both['device_category'] = d['device_category']
exp_test_set_both['ad_network'] = d['ad_network']

In [251]:
exp_train_set_both['ad_network'] = exp_train_set_both['ad_network'].fillna('organic')
exp_test_set_both['ad_network'] = exp_test_set_both['ad_network'].fillna('organic')

In [252]:
columns_to_encode = ['platform', 'device_category', 'ad_network']

In [253]:
transformed_train_set_df = pd.get_dummies(exp_train_set_both, columns=columns_to_encode)
transformed_test_set_df = pd.get_dummies(exp_test_set_both, columns=columns_to_encode)

In [258]:
transformed_test_set_df = transformed_test_set_df.reindex(columns=transformed_train_set_df.columns, fill_value=0)

In [260]:
boolean_columns = transformed_train_set_df.select_dtypes(include='bool').columns
transformed_train_set_df[boolean_columns] = transformed_train_set_df[boolean_columns].astype(int)

boolean_columns = transformed_test_set_df.select_dtypes(include='bool').columns
transformed_test_set_df[boolean_columns] = transformed_test_set_df[boolean_columns].astype(int)

In [269]:
model = LinearRegression(n_jobs=N_JOBS)

In [288]:
# scaling level duration columns, relative first open and level duration sum
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

transformed_train_set_df['Level_Duration_Sum'] = scaler.fit_transform(transformed_train_set_df[['Level_Duration_Sum']])
transformed_train_set_df['Relative_First_Open'] = scaler.fit_transform(transformed_train_set_df[['Relative_First_Open']])
transformed_train_set_df[level_duration_cols] = scaler.fit_transform(transformed_train_set_df[level_duration_cols])

transformed_test_set_df['Level_Duration_Sum'] = scaler.fit_transform(transformed_test_set_df[['Level_Duration_Sum']])
transformed_test_set_df['Relative_First_Open'] = scaler.fit_transform(transformed_test_set_df[['Relative_First_Open']])
transformed_test_set_df[level_duration_cols] = scaler.fit_transform(transformed_test_set_df[level_duration_cols])

In [295]:
model.fit(transformed_train_set_df, train_target)
yhat = model.predict(transformed_test_set_df)

c:\Users\Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [296]:
yhat

array([0.01718631, 0.01968131, 0.01846723, ..., 1.84851701, 0.00292521,
       1.20662042])

In [297]:
submission = test_IDs.copy()
submission['TARGET'] = yhat

In [298]:
submission

,ID,TARGET
0,878594,0.017186
1,878595,0.019681
2,878596,0.018467
3,878597,0.408775
4,878598,0.044389
...,...,...
585725,1464319,0.020782
585726,1464320,3.997857
585727,1464321,1.848517
585728,1464322,0.002925


In [299]:
submission.to_csv('submission.csv', index=False)

In [300]:
submission.loc[submission['TARGET'] < 0, 'TARGET'] = 0

In [301]:
submission.to_csv('submission_final.csv', index=False)